In [6]:
### Building a rule-based Chatbot

In [7]:
import nltk
from nltk.chat.util import Chat, reflections

In [8]:
print(Chat)

<class 'nltk.chat.util.Chat'>


In [9]:
print(reflections)

{'i am': 'you are', 'i was': 'you were', 'i': 'you', "i'm": 'you are', "i'd": 'you would', "i've": 'you have', "i'll": 'you will', 'my': 'your', 'you are': 'I am', 'you were': 'I was', "you've": 'I have', "you'll": 'I will', 'your': 'my', 'yours': 'mine', 'you': 'me', 'me': 'you'}


In [51]:
set_pairs = [
    [
        r"my name is (.*)" ,
        ["Hello %1, How are you today?",]
    ],
    [
        r"hi|hay|hello" ,
        ["Hello", "Hey there",]
    ],
    [
        r"what is your name ?",
        ["You can call me a chatbot?",]
    ],
    [
        r"how are you ?",
        ["I am fine, thank you! How can I help you?",]
    ],  
    [
        r"how can I help you?",
        ["I am looking for online guides and courses to learn data science, can you suggest some good resources?",]
    ],
    [
        r"I'm(.*)doing good",
        ["That's great to hear", "How can I help you?:)",]
    ],
    [
        r"I am looking for online guides and courses to learn data science, can you suggest some good resources?",
        ["Coding Ninjas is a great option to learn data science. You can check their website at https://www.codingninjas.com/",],
    ],  
    [
        r"thanks for the suggestion. do they have great authors and instructors?",
        ["Yes, they have the world class best authors, that is their strength:)",]
    ],
    [
        r"(.*) thank you so much , that was helpful",
        ["I am happy to help", "No problem, you're welcome",]
    ],
    [
        r"It was nice talking to you. See you soon!",
        ["Bye, take care. See you soon :)", "It was nice talking to you. See you soon :)",]
    ],

]


In [52]:
def chatbot() :
    print("Hi, I am a rule-based chatbot! What can I help you with?")
chatbot()    

Hi, I am a rule-based chatbot! What can I help you with?


In [53]:
chat = Chat(set_pairs, reflections)
print(chat)

In [54]:
chat.converse()
if __name__ == "__main__":
    chatbot()

>Hi
Hey there
>My name is Ria
Hello ria, How are you today?
>What is your name?
You can call me a chatbot?
>How are you?
I am fine, thank you! How can I help you?
>I am looking for online guides and courses to learn data science, can you suggest some good resources?
Coding Ninjas is a great option to learn data science. You can check their website at https://www.codingninjas.com/
>thanks for the suggestion. do they have great authors and instructors?
Yes, they have the world class best authors, that is their strength:)
>Chatbot thank you so much , that was helpful
No problem, you're welcome
>It was nice talking to you. See you soon!
None


KeyboardInterrupt: Interrupted by user

In [ ]:
## Building a retrieval based chatbot!

In [55]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import requests
import re

import warnings
warnings.filterwarnings = False

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [63]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [64]:
r = requests.get('https://en.wikipedia.org/wiki/Cuisine')
raw_html = r.text
r.text

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Cuisine - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";var cookie=document.cookie.match(/(?:^|; )enwikimwclientpreferences=([^;

In [66]:
##cleaning up
corpus_html = bs.BeautifulSoup(raw_html)

##extracting paragraphs from the html
corpus_paras = corpus_html.find_all('p')
corpus_text = ''

##concatenating all the paras
for para in corpus_paras:
    corpus_text += para.text

##lowering the text
corpus_text = corpus_text.lower()

In [67]:
corpus_text

'\na cuisine  is a style of cooking characterized by distinctive ingredients, techniques and dishes, and usually associated with a specific culture or geographic region. regional food preparation techniques, customs, and ingredients  combine to enable dishes unique to a region.[1]\nused in english since the late 18th century, the word cuisine – meaning manner or style of cooking – is borrowed from the french for "style of cooking," as originally derived from latin coquere "to cook".[2]\na cuisine is partly determined by ingredients that are available locally or through trade. regional ingredients are developed and commonly contribute to a regional or national cuisine, such as japanese rice in japanese cuisine.\nreligious food laws can also exercise an influence on cuisine, such as indian cuisine and hinduism that is mainly lacto-vegetarian (avoiding meat and eggs) due to sacred animal worship. sikhism in punjabi cuisine, buddhism in east asian cuisine, christianity in european cuisine,

In [68]:
##getting rid of all the empty spaces and special characters
corpus_text = re.sub(r'\[[0-9]*\]', ' ', corpus_text)
corpus_text = re.sub(r'\s+' , ' ' , corpus_text)

In [69]:
##converting text into sentences and word tokens
corpus_sentences = nltk.sent_tokenize(corpus_text)
corpus_words = nltk.word_tokenize(corpus_text)

In [70]:
corpus_sentences

[' a cuisine is a style of cooking characterized by distinctive ingredients, techniques and dishes, and usually associated with a specific culture or geographic region.',
 'regional food preparation techniques, customs, and ingredients combine to enable dishes unique to a region.',
 'used in english since the late 18th century, the word cuisine – meaning manner or style of cooking – is borrowed from the french for "style of cooking," as originally derived from latin coquere "to cook".',
 'a cuisine is partly determined by ingredients that are available locally or through trade.',
 'regional ingredients are developed and commonly contribute to a regional or national cuisine, such as japanese rice in japanese cuisine.',
 'religious food laws can also exercise an influence on cuisine, such as indian cuisine and hinduism that is mainly lacto-vegetarian (avoiding meat and eggs) due to sacred animal worship.',
 'sikhism in punjabi cuisine, buddhism in east asian cuisine, christianity in euro

In [71]:
corpus_words

['a',
 'cuisine',
 'is',
 'a',
 'style',
 'of',
 'cooking',
 'characterized',
 'by',
 'distinctive',
 'ingredients',
 ',',
 'techniques',
 'and',
 'dishes',
 ',',
 'and',
 'usually',
 'associated',
 'with',
 'a',
 'specific',
 'culture',
 'or',
 'geographic',
 'region',
 '.',
 'regional',
 'food',
 'preparation',
 'techniques',
 ',',
 'customs',
 ',',
 'and',
 'ingredients',
 'combine',
 'to',
 'enable',
 'dishes',
 'unique',
 'to',
 'a',
 'region',
 '.',
 'used',
 'in',
 'english',
 'since',
 'the',
 'late',
 '18th',
 'century',
 ',',
 'the',
 'word',
 'cuisine',
 '–',
 'meaning',
 'manner',
 'or',
 'style',
 'of',
 'cooking',
 '–',
 'is',
 'borrowed',
 'from',
 'the',
 'french',
 'for',
 '``',
 'style',
 'of',
 'cooking',
 ',',
 "''",
 'as',
 'originally',
 'derived',
 'from',
 'latin',
 'coquere',
 '``',
 'to',
 'cook',
 "''",
 '.',
 'a',
 'cuisine',
 'is',
 'partly',
 'determined',
 'by',
 'ingredients',
 'that',
 'are',
 'available',
 'locally',
 'or',
 'through',
 'trade',
 '.',


In [72]:
greeting_inputs = ("hey", "good morning", "good evening", "morning", "hi", "whatsup")
greeting_responses = ["hey", "hey hows you?","nods", "hello,how you doing", "hello", "Welcome,I am good" ]
def greet_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [74]:
wn_lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_corpus(tokens):
    return[wn_lemmatizer.lemmatize(token) for token in tokens]
punct_removal_dict = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return lemmatize_corpus(nltk.word_tokenize(document.lower().translate(punct_removal_dict)))

In [97]:
punct_removal_dict = dict((ord(punctuation), None) for punctuation in string.punctuation)
punct_removal_dict

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [104]:
def respond(user_input):
    '''
    generating response after
    '''
    bot_response = ''
    corpus_sentences.append(user_input)
    
    ##vectorizing the processed text
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words ='english')
    corpus_word_Vectors = word_vectorizer.fit_transform(corpus_sentences)
    
    cos_sim_vectors = cosine_similarity(corpus_word_Vectors[-1], corpus_word_Vectors)
    similar_response_idx = cos_sim_vectors.argsort()[0][-2]
    
    matched_vector = cos_sim_vectors.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]
    
    if vector_matched == 0:
        bot_response = bot_response + "I am sorry, what is it, again?"
        return bot_response
    else:
        bot_response = bot_response + corpus_sentences[similar_response_idx]
        return bot_response

In [105]:
chat = True
print("Hello, What do you want to learn about Cuisines today?")
while(chat == True):
    user_query = input()
    user_query = user_query.lower()
    if user_query != 'quit':
        if user_query == 'thanks' or user_query == 'thank you':
            chat == False
            print("CuisineBot: You are welcome!")
        else:
            if greet_response(user_query)!= None:
                print("CuisineBot: "+ greet_response(user_query)) 
            else:
                print("CuisineBot:", end="")
                print(respond(user_query))
                corpus_sentences.remove(user_query)
    else:
        chat = False
        print("CusineBot: Good bye!")

Hello, What do you want to learn about Cuisines today?
Hi
CuisineBot: nods
cuisine
CuisineBot:cuisine
pizza
CuisineBot:an english sunday roast with roast beef, roast potatoes, vegetables and yorkshire pudding traditional pizza from napoli: originally italian dish german sausages and cheese beef stroganoff, a russian dish.
beer
CuisineBot:buffalo wings with blue cheese dressing, served with lager beer.
mexican
CuisineBot:mexican
ingredients
CuisineBot:a cuisine is partly determined by ingredients that are available locally or through trade.
culinary culture
CuisineBot:some factors that have an influence on a region's cuisine include the area's climate, the trade among different countries, religious or sumptuary laws and culinary culture exchange.
south american
CuisineBot:the regional cuisines are north american cuisine, mexican cuisine, central american cuisine, south american cuisine, and caribbean cuisine.
tacos
CuisineBot:tacos filled with several meat types, mainly beef, chicken an

KeyboardInterrupt: Interrupted by user